
1. 자연어 기초 실습
2. Huggingface
3. PEFT (LoRA)
4. ChatGPT 활용 실습

# 1. 자연어 기초 실습


## (1) 토큰화 (Tokenization)
- 토큰화(Tokenization) : 주어진 문장을 일정한 단위로 나누는 것. 주로 단어/문자/형태소 단위로 나눔.
  - huggingface에서 Pretrained Model을 사용할 땐 그 모델의 vocabulary에 맞추서 토큰화 해야 함.
    - vocabulary : 중복을 제거한 텍스트의 총 단어의 집합(set)
- 영어 토큰화 도구로는 spaCy와 NLTK가 있음.


In [ ]:
# 영어 토큰화 (spacy 사용하기)
en_text = "A Dog Run back corner near spare bedrooms"

import spacy
spacy_en = spacy.load('en_core_web_sm')

def tokenize(text):
  return [tok.text for tok in spacy_en.tokenizer(en_text)]

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
spacy_en.tokenizer(en_text)

A Dog Run back corner near spare bedrooms

In [ ]:
tokenize(en_text)

['A', 'Dog', 'Run', 'back', 'corner', 'near', 'spare', 'bedrooms']

In [ ]:
# 영어 토큰화 (nltk 사용하기)
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
print(word_tokenize(en_text))

['A', 'Dog', 'Run', 'back', 'corner', 'near', 'spare', 'bedrooms']


In [ ]:
# 영어 토큰화 (띄어쓰기 기준)
print(en_text.split())

['A', 'Dog', 'Run', 'back', 'corner', 'near', 'spare', 'bedrooms']


In [ ]:
# 영어 토큰화 (문자)
print(list(en_text))

In [ ]:
# 한국어 토큰화 (띄어쓰기)
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사왔어"
print(kor_text.split())     # 영어와 달리 이렇게 토큰화하면 조사와 결합한 단어들은 전부 다른 단어들로 인식됨 -> vocabulary가 불필요하게 커짐.

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사왔어']


In [ ]:
# 한국어 토큰화 (형태소)
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
from konlpy.tag import Mecab
tokenizer = Mecab()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이', '라는', '글', '을', '봤', '어', '.', '그래서', '오늘', '사과', '를', '먹', '으려고', '했', '는데', '사과', '가', '썩', '어서', '슈퍼', '에', '가', '서', '사과', '랑', '오렌지', '사', '왔', '어']


## (2) Vocabulary 생성

In [ ]:
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
data = pd.read_table('ratings.txt') # 데이터프레임에 저장
data[:10]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁2 나오나요? 나오면 1빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [ ]:
print('전체 샘플의 수 : {}'.format(len(data)))
sample_data = data[:100] # 임의로 100개만 저장

전체 샘플의 수 : 200000


In [ ]:
# 정규표현식을 사용해서 정제
sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
sample_data[:10]

<ipython-input-22-b95157ca7842>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-22-b95157ca7842>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['document'] = sample_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,4655635,폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고,1
3,9251303,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,2190435,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,9279041,완전 감동입니다 다시봐도 감동,1
7,7865729,개들의 전쟁 나오나요 나오면 빠로 보고 싶음,1
8,7477618,굿,1
9,9250537,바보가 아니라 병 쉰 인듯,1


In [ ]:
# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
tokenizer = Mecab()
tokenized=[]
for sentence in sample_data['document']:
    temp = tokenizer.morphs(sentence) # 토큰화
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    tokenized.append(temp)

print(tokenized[:10])

[['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], ['디자인', '을', '배우', '학생', '외국', '디자이너', '그', '일군', '전통', '을', '통해', '발전', '해', '문화', '산업', '부러웠', '는데', '사실', '우리', '나라', '에서', '그', '어려운', '시절', '끝', '까지', '열정', '을', '지킨', '노라노', '같', '전통', '있', '어', '저', '같', '사람', '꿈', '을', '꾸', '고', '이뤄나갈', '수', '있', '다는', '것', '감사', '합니다'], ['폴리스', '스토리', '시리즈', '부터', '뉴', '까지', '버릴', '께', '하나', '없', '음', '최고'], ['연기', '진짜', '개', '쩔', '구나', '지루', '할거', '라고', '생각', '했', '는데', '몰입', '해서', '봤', '다', '그래', '이런', '게', '진짜', '영화', '지'], ['안개', '자욱', '밤하늘', '떠', '있', '초승달', '같', '영화'], ['사랑', '을', '해', '본', '사람', '라면', '처음', '부터', '끝', '까지', '웃', '을', '수', '있', '영화'], ['완전', '감동', '입니다', '다시', '봐도', '감동'], ['개', '전쟁', '나오', '나요', '나오', '면', '빠', '로', '보', '고', '싶', '음'], ['굿'], ['바보', '아니', '라', '병', '쉰', '인', '듯']]


In [ ]:
vocab = FreqDist(np.hstack(tokenized))    # FreqDist: nltk에서 지원하는 빈도수 계산 도구 {단어 : 빈도수}
print('단어 집합의 크기 : {}'.format(len(vocab)))

단어 집합의 크기 : 664


In [ ]:
vocab['재밌']   # 빈도수 출력

10

In [ ]:
vocab_size = 500
# 상위 vocab_size개의 단어만 보존 (등장 빈도수 상위 500개)
vocab = vocab.most_common(vocab_size)
print('단어 집합의 크기 : {}'.format(len(vocab)))

단어 집합의 크기 : 500


# 2. huggingface

In [ ]:
!pip install transformers

In [ ]:
# pipeline 사용해서 inference 해보기
from transformers import AutoModelForMaskedLM, AutoTokenizer

MODEL_NAME = 'bert-base-multilingual-cased'
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

text = "이순신은 [MASK] 중기의 무신이다."

tokenizer.tokenize(text)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

['이', '##순', '##신', '##은', '[MASK]', '중', '##기의', '무', '##신', '##이다', '.']

In [ ]:
from transformers import pipeline

kor_mask_fill = pipeline(task='fill-mask', model=model, tokenizer=tokenizer)

In [ ]:

text = "이순신은 [MASK] 중기의 무신이다."

kor_mask_fill("이순신은 [MASK] 중기의 무신이다.")

[{'score': 0.8747133612632751,
  'token': 59906,
  'token_str': '조선',
  'sequence': '이순신은 조선 중기의 무신이다.'},
 {'score': 0.06436370313167572,
  'token': 9751,
  'token_str': '청',
  'sequence': '이순신은 청 중기의 무신이다.'},
 {'score': 0.01095485407859087,
  'token': 9665,
  'token_str': '전',
  'sequence': '이순신은 전 중기의 무신이다.'},
 {'score': 0.004647126886993647,
  'token': 22200,
  'token_str': '##종',
  'sequence': '이순신은종 중기의 무신이다.'},
 {'score': 0.00361065031029284,
  'token': 12310,
  'token_str': '##기',
  'sequence': '이순신은기 중기의 무신이다.'}]

In [ ]:
# 사전학습된 토크나이저 로드
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
sequence = "In a hole in the ground there lived a hobbit."
print(tokenizer(sequence))

{'input_ids': [101, 1999, 1037, 4920, 1999, 1996, 2598, 2045, 2973, 1037, 7570, 10322, 4183, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(
    "ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition"
)

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv2-base-uncased")

preprocessor_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
# auto model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased")

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 3. PEFT
- Parameter Efficient FineTuning
- LoRA

- pretrained model을 finetuning해서 downstream task를 푸는데 필요한 정보는 일부
- 기존 연구들을 기반으로, 필요한 정보는 intrinsic dimension에 있을 것이라고 판단했음.
- intrinsic dimension을 찾기 위해 Low-Rank Space로 변환하는 방식을 사용하면더 적은 파라미터로도 같은 성능을 낼 수 있을 것이라고 판단.
- GPT-3의 경우에, 12,888에 해당하는 원본(full-rank) 정보를 r=1,2 정도의 저차원에서 표현 가능하다는 것을 밝힘.

In [ ]:
!git clone -q https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
!pip uninstall peft -y -q
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

/content/alpaca-lora
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.3 MB

In [ ]:
!wget -q https://github.com/Beomi/KoAlpaca/raw/main/ko_alpaca_data.json

import json

with open('ko_alpaca_data.json', 'r') as f:
  dataset = json.load(f)

dataset[:5]

[{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
  'input': '',
  'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'},
 {'instruction': '세 가지 기본 색은 무엇인가요?',
  'input': '',
  'output': '기본 색은 빨강, 파랑, 노랑입니다.'},
 {'instruction': '원자의 구조를 설명하세요.',
  'input': '',
  'output': '원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있고 전자는 주변에 있습니다.'},
 {'instruction': '대기 오염을 어떻게 줄일 수 있나요?',
  'input': '',
  'output': '대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.'},
 {'instruction': '어려운 결정을 내려야 했던 때를 설명하세요.',
  'input': '',
  'output': '제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.'}]

In [ ]:
import json

prompt_template = {
    "description": "Alpaca-LoRA Custom 템플릿",
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response:\n"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response:\n"
    ),
    "response_split": "### Response:",
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

In [ ]:
!python finetune.py \
    --base_model 'baffo32/decapoda-research-llama-7B-hf' \
    --data_path 'ko_alpaca_data.json' \
    --output_dir './output' \
    --num_epochs 5 \
    --learning_rate 5e-4 \
    --val_set_size 2000 \
    --batch_size 512 \
    --micro_batch_size 16 \
    --prompt_template_name 'custom'

Training Alpaca-LoRA model with params:
base_model: baffo32/decapoda-research-llama-7B-hf
data_path: ko_alpaca_data.json
output_dir: ./output
batch_size: 512
micro_batch_size: 16
num_epochs: 5
learning_rate: 0.0005
cutoff_len: 256
val_set_size: 2000
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: False
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: custom

config.json: 100% 428/428 [00:00<00:00, 1.55MB/s]
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
pytorch_model.bin.index.json: 100% 25.5k/25.5k [00:00<00:00, 29.4MB/s]
pytorch_model-00001-of-00033.bin:   0% 0.00/405M [00:00<?, ?B/s]
pytorch_model-00001-of-00033.bin:   3% 10.5M/405M [00:00<00:06, 65.6MB/s]
pytorch_model-00001-of-00

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/LLaMa-Alpaca-LoRA
!cp -a output /content/drive/MyDrive/LLaMa-Alpaca-LoRA

In [ ]:
# test on gradio
!python generate.py \
    --base_model 'dbaffo32/decapoda-research-llama-7B-hf' \
    --lora_weights '/content/drive/MyDrive/LLaMa-Alpaca-LoRA/output' \
    --prompt_template 'custom' \
    --share_gradio

# 4. ChatGPT 활용 실습

In [ ]:
import openai
from tqdm import tqdm



# 1개의 summary
def get_abstractive_summary(context) -> str:
    prompt = f"{context} Please provide a concise and abstractive summary of the following text: "
    response = call_chatgpt(prompt)
    result = response['choices'][0]['message']['content']

    return result


# 5개의 summary
def get_abstractive_summaries(context) -> list:
    prompt = f"{context} Please extract the most important sentences or phrases from the following text to create 5 summaries: "
    response = call_chatgpt(prompt)
    result = response['choices'][0]['message']['content']

    # 번호표 제거
    result = [" ".join(sum.split(" ")[1:]) for sum in result.split('\n')]

    return result


import time

def call_chatgpt(prompt) -> dict:
    openai.api_key = ""
    retries = 3
    while retries > 0:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{'role': 'user', 'content': prompt}],
                temperature=1.5  # option
            )

            return response

        except Exception as e:
            if e:
                print(e)
                print('Timeout error, retrying...')
                retries -= 1
                time.sleep(5)
            else:
                raise e

    print("API is not responding, moving on...")
    bad_api = 'x'
    return {bad_api : bad_api}




if __name__ == "__main__":

    # dataset_type = "spoken_squad_train"        # ami, spoken_squad
    dataset_type = "spoken_squad_test"
    contexts = []
    filenames = []
    dup_count = 0

    if dataset_type == "ami":
        pass
    if "spoken_squad" in dataset_type :
        is_train = dataset_type.split('_')[-1]
        with open(f'/mnt/ssd03_4tb/juny/Spoken-SQuAD_audio/spoken_squad_manifest/{is_train}.ltr') as file:
            for line in tqdm(file, desc="Parsing Manifest..."):
                context, _, _, filename = line.strip('\n').split('\t')         # context, question, answer, filenames
                if context not in contexts:
                    contexts.append(context)
                    filenames.append(filename)
                else:
                    dup_count += 1

            print("Duplicated Context : ", dup_count)
    else:
        print("INVALID DATASET TYPE")
        exit()

    abssum_file = open(f"{dataset_type}_abssum_aug.tsv", 'w')
    extsum_file = open(f"{dataset_type}_extsum_aug.tsv", 'w')
    paraph_file = open(f"{dataset_type}_paraph_aug.tsv", 'w')
    abssum_short5_file = open(f"{dataset_type}_abssum_short5_aug.tsv", 'w')
    extsum_short5_file = open(f"{dataset_type}_extsum_short5_aug.tsv", 'w')

    for context, filename in tqdm(zip(contexts, filenames), desc="Generate Summaries..."):
        abssum = get_abstractive_summary(context)
        extsum = get_extractive_summary(context)
        paraph = do_paraphrasing(context)

        abssum_5 = get_abstractive_summaries(context)
        extsum_5 = get_extractive_summaries(context)

        print(f"{abssum}\t{filename}", file=abssum_file)
        print(f"{extsum}\t{filename}", file=extsum_file)
        print(f"{paraph}\t{filename}", file=paraph_file)

        for abs, ext in zip(abssum_5, extsum_5):
            print(f"{abs}\t{filename}", file=abssum_short5_file)
            print(f"{ext}\t{filename}", file=extsum_short5_file)

    abssum_file.close()
    extsum_file.close()
    paraph_file.close()
    abssum_short5_file.close()
    extsum_short5_file.close()


